In [ ]:
import random

#returns a random Elliptic curve over a field GF(2^p)
#primeOrd forces returned EC to be of prime order
def generateRandomEC(p, primeOrd=False):
    T.<a> = GF(2**p)
    while True:
        coefs = []
        for i in range(0,5):
            coefs.append( T.random_element() )
        try:
            E = EllipticCurve(T, coefs)
            if primeOrd == False or is_prime(E.order()):
                break
        except ArithmeticError: #if E singular
            pass
    print(E)
    return E

#returns a random multiple of a base point P and order of subgroup <P>
def generateRandomPoint(P):
    orderP=P.order()
    k=int(1+(orderP-1)*random.random()) #k in [1, orderP-1]
    Q=k*P
    return Q, orderP

In [10]:
#Individual step of Pohlig-Hellman algorithm
#Solves DLP in a subgroup of order 'orderP'
#https://courses.fit.cvut.cz/MI-MKY/media/lectures/mi-mky-poznamky-v17.pdf (page 81 PDF)
#Pi - generator of the subgroup, Qi = ki*Pi, order of Pi is orderP, returns ki
#Splits ki = x0 + p*x1 + p^2*x2 + p^(e-1)*x_(e-1) and solves 'e'-times DLP in a subgroup of order 'orderP'
def PH_reduction(Qi, Pi, orderP, **kwargs): 
    invPi = -Pi #additive inverse
    p = prime_factors(orderP)[0]
    e = orderP.valuation(p)
    
    Pi = (p^(e-1))*Pi
    ki = 0 #ki = x0 + p*x1 + p^2*x2 + ...
    for i in srange(0, e):
        tempQ = (p^(e-1-i))*(Qi + ki*invPi)
        ki += (p^i)*discrete_log_rho(tempQ, Pi, operation="+")
    return ki


#Implements Pohlig-Hellman algorithm
#https://courses.fit.cvut.cz/MI-MKY/media/lectures/mi-mky-poznamky-v17.pdf (page 81 PDF)
#Solves Q = k*P, returns k
#P generates a subgroup of order orderP
#DLP_Solve is a function solving ECDLP in a subgroup of order p^e, where 'p' is a prime number
def Pohlig_Hellman_additive(Q, P, orderP, DLP_Solve = PH_reduction, **kwargs):
    print("Factorization of orderP: {} = {}.".format(orderP, orderP.factor()))
    print("Order of E = <P> is {}.\nBit security (log2 of the biggest prime_factor of orderP): {:4.1f}"\
      .format(orderP, float(log(max(prime_factors(orderP)))/log2)))
    
    ki = [] #list of individual DLP results
    mi = [] #list of moduli
    for p in prime_factors(orderP): #prime factor p
        e = orderP.valuation(p) #multiplicity of p in orderP
        mi.append(p^e) 
        pei = Integer(orderP/mi[-1])
        ki.append(DLP_Solve(pei*Q, pei*P, mi[-1], **kwargs)) #solves ECDLP in a subgroup <pei*P>
    return Integer(CRT(ki, mi))


In [11]:
import random
#Generates a base of random distinct points (a*P + b*Q), size depends on 'm'
#Based on (Algorithm 4.1) https://eprint.iacr.org/2017/1262.pdf page 10
#Input: P - base point
#       Q = k*P
#       orderP is order of group <P>
#Returns:   coord, factorBase, res
#           coord is a list of (a_i, b_i) coefficients
#           factorBase is a list of factorBase points (a_i*P + b_i*Q)
#           If res != -1 it's a solution to the ECDLP => res*P = Q
def buildFactorBase(Q, P, orderP, m = 3):
    factorBaseSize = ceil(orderP^(1/m)) 
    factorBase = []
    coord = []
    currentSize = 0
    res = -1
    while currentSize < factorBaseSize:
        a = int(random.random()*orderP) #a,b in [0, orderE - 1]
        b = int(random.random()*orderP)
        candidate=a*P+b*Q
        if candidate not in factorBase:
            factorBase.append(candidate)
            coord.append((a,b))
            currentSize += 1
        else: #we can try to solve the whole problem here
            c, d = coord[factorBase.index(candidate)]
            
            #relationship: a*P + b*Q = c*P + d*Q => (a-c)*(d-b)^(-1)*P=Q
            try:
                res = Integer(mod((a - c)*inverse_mod(d - b, orderP), orderP))
                break
            except ZeroDivisionError: #inverse of 'b' mod 'orderP' does not exist
                pass
                
    return coord, factorBase, res


#Multi-index class, dimension is dim
#[i_1, i_2, ..., i_dim]
class MultiIndex:
    def __init__(self, dim, baseSize):
        self.dim = dim
        self.baseSize = baseSize
        self.index = [0 for i in srange(0, self.dim)]
        
    def dbgPrint(self):
        print("Dim: {}, baseSize: {}, index: {}.".format(self.dim, self.baseSize, self.index))

    def getIndex(self):
        return self.index
    
    #returns a multi-index, -1 means the index is out of bounds
    def nextIndex(self):
        isDone = True
        for i in srange(self.dim - 1, -1, -1):
            if (self.index[i] + 1) < self.baseSize:
                isDone = False
                break
        if isDone == True:
            return [-1]
        
        self.index[i] += 1
        for k in srange(i + 1, self.dim):
            self.index[k] = self.index[i]
        return self.index

In [ ]:
import time

def sumInF(Q, P, orderP, **kwargs):
    m = 3
    maxAttempts = 16
    for key, value in kwargs.items():
        if key == "m":
            m = Integer(value)
        elif key == "maxAttempts":
            maxAttempts = Integer(value)

    for attempt in srange(0, maxAttempts):
        coord, factorBase, res = buildFactorBase(Q, P, orderP, m)
        if res != -1:
            return res

        length = len(factorBase)
        print("Attempt: {}, EC order: {}, factor base size: {}.".format(attempt+1, orderP, length))
        dim = m - 1
        multiIndex = MultiIndex(dim, length)
        index = multiIndex.getIndex()

        while index[0] != -1:
            dp = [factorBase[i] for i in index]
            for v in VectorSpace(GF(2), dim):
                test = sum(-dp[k] if v[k] else dp[k] for k in srange(0, dim))

                if test in factorBase:
                    baseId = factorBase.index(test)

                    b = coord[baseId][1] + sum(coord[index[k]][1] if v[k] 
                                              else -coord[index[k]][1] for k in srange(0,dim))
                    a = -coord[baseId][0] + sum(-coord[index[k]][0] if v[k] 
                                              else coord[index[k]][0] for k in srange(0,dim))
                    try:
                        return Integer(mod(a*inverse_mod(b, orderP), orderP))
                    except ZeroDivisionError: #inverse of 'b' mod 'orderP' does not exist
                        pass
            index = multiIndex.nextIndex()
    return -1

def bruteForce(P, Q, orderP):
    for i in srange(0, orderP):
        if (i*P == Q):
            return i
    raise ValueError #no solution to this DLP

#e is not used
def sum2inF(Q, P, orderP, **kwargs):
    maxAttempts = 16
    for key, value in kwargs.items():
        if key == "maxAttempts":
            maxAttempts = value
            break
            
    for attempt in srange(0, maxAttempts):
        coord, factorBase, res = buildFactorBase(Q, P, orderP)
        if res != -1:
            return res
        
        length = len(factorBase)
        print("Attempt: {}, EC order: {}, factor base size: {}.".format(attempt+1, orderP, length))
        for i in srange(0, length):
            for j in srange(i, length):
                ids = [i, j]
                for v in VectorSpace(GF(2), 2):
                    test = sum(-factorBase[ids[k]] if v[k] else factorBase[ids[k]] for k in [0,1]);

                    if test in factorBase:
                        baseId = factorBase.index(test)

                        b = coord[baseId][1] + sum(coord[ids[k]][1] if v[k] 
                                                   else -coord[ids[k]][1] for k in [0,1])

                        a = -coord[baseId][0] + sum(-coord[ids[k]][0] if v[k] 
                                                    else coord[ids[k]][0] for k in [0,1])

                        try:
                            res = Integer(mod(a*inverse_mod(b, orderP), orderP))
                            return res
                        except ZeroDivisionError: #inverse of 'b' mod 'orderP' does not exist
                            pass
    raise ValueError #if logarithm wasnt found

#import timeit
#print(timeit.timeit('sum2inF(coord, factorBase, orderP)', 'from __main__ import sum2inF, coord, factorBase, orderP', number=100))
#print(timeit.timeit('sumInF(coord, factorBase, orderP, 3)', 'from __main__ import sumInF, coord, factorBase, orderP', number=100))
#prime
#11 | 13 | 17 | 19 | 23 | 29 | 31 | 37 | 41 | 43 | 47 | 53 | 59 | 61 | 67 | 71 | 73 | 79 | 83 | 89 | 97
p = 31
E = generateRandomEC(p)
P = E.gen(0) #base point P
Q, orderP = generateRandomPoint(P)
print("Order of P is {}.".format(orderP))

print("Factorization of Elliptic curve order: {} = {}".format(orderP, orderP.factor()))

tm = time.time()
r1 = discrete_log(Q,P, E.order(), operation='+')
print("Sage: It took {:4.4f} seconds. Result isValid: {}.\n"\
       .format((time.time()-tm), bool(r1*P == Q)))

tm = time.time()
r1 = Pohlig_Hellman_additive(Q,P, E.order())
print("PH-rho: It took {:4.4f} seconds. Result isValid: {}.\n"\
       .format((time.time()-tm), bool(r1*P == Q)))

# start = time.time()
# res = bruteForce(P, Q, orderP)
# print("It took {:4.4f} seconds. Brute force. ".format(time.time()-start))
      
tm = time.time()
while True:
    res = Pohlig_Hellman_additive(Q, P, orderP, sum2inF)

    if res != -1:
        break
tm = time.time() - tm
print("Sum2F: It took {:4.4f} seconds. Result isValid: {}.\n"\
      .format(tm, bool(res*P == Q)))
assert(res*P == Q)

for m in srange(3, 7):
    tm = time.time()
    kwargs = {"m" : m, "maxAttempts" : 25}
    while True:
        res=Pohlig_Hellman_additive(Q, P, orderP, sumInF, **kwargs)
        if res != -1 or attempts > 20:
            break
    tm = time.time() - tm
    print("For m = {} it took {:4.4f} seconds. Result isValid: {}.\n"\
          .format(m, tm, bool(res*P == Q)))

Elliptic Curve defined by y^2 + (a^28+a^24+a^20+a^19+a^18+a^17+a^16+a^14+a^13+a^2)*x*y + (a^30+a^29+a^28+a^26+a^24+a^23+a^21+a^19+a^17+a^16+a^15+a^14+a^13+a^10+a^9+a^8+a^7+a^6+a^4+a)*y = x^3 + (a^26+a^25+a^24+a^22+a^20+a^16+a^14+a^8+a^7+a+1)*x^2 + (a^30+a^29+a^28+a^26+a^25+a^24+a^21+a^18+a^14+a^13+a^8+a^6+a^4+a^3+a+1)*x + (a^30+a^27+a^26+a^25+a^24+a^19+a^17+a^16+a^5+a^4+a^3+a+1) over Finite Field in a of size 2^31
Order of P is 2147534778.
Factorization of Elliptic curve order: 2147534778 = 2 * 3 * 357922463
Sage: It took 2.7265 seconds. Result isValid: True.

Factorization of orderP: 2147534778 = 2 * 3 * 357922463.
Order of E = <P> is 2147534778.
Bit security (log2 of the biggest prime_factor of orderP): 28.4
PH-rho: It took 2.4426 seconds. Result isValid: True.

Factorization of orderP: 2147534778 = 2 * 3 * 357922463.
Order of E = <P> is 2147534778.
Bit security (log2 of the biggest prime_factor of orderP): 28.4
Attempt: 1, EC order: 2, factor base size: 2.
Attempt: 2, EC order: 2, f

In [ ]:
#https://www.maths.unsw.edu.au/sites/default/files/mandyseetthesis.pdf (page 76)
########################################################################
# Author: Dr. D. Kohel, M. Seet #
# #
# SAGE Mathematical Software, Version 2.6, http://www.sagemath.org #
# #
########################################################################

In [ ]:
class MultiIndex:
    def __init__(self, m, orderP):
        self.dim = m - 1
        self.orderP = orderP
        self.index = [0 for i in range(0, self.dim)]
        
    def dbgPrint(self):
        print("Dim: {}, order: {}, index: {}.".format(self.dim, self.orderP, self.index))
        
    #returns a multi-index, -1 means the index is out of bounds
    def nextIndex(self):
        isDone = True
        for i in range(self.dim - 1, -1, -1):
            if (self.index[i] + 1) < self.orderP:
                isDone = False
                break
        if isDone == True:
            return [-1]
        
        self.index[i] += 1
        for k in range(i + 1, self.dim):
            self.index[k] = self.index[i]
        return self.index

In [ ]:
Q=GF(2^5)
print(Q.random_element())

In [ ]:
print(i)